# Assignment 7

## Submit as an HTML file

### Print your name below

In [1]:
print("Susu Zhao")

Susu Zhao


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [41]:
# Write your answer here:

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [59]:
results = pd.read_csv("data/results.csv")
display(results.iloc[0:5,:])

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [44]:
# Write your answer here
results.dtypes
# columns that contain "object"/string data may need conversion (number, position, positionText, etc.)
list_old = ['\\N']
list_new = [np.nan]

results["race_time_ms"] = results["milliseconds"].replace(list_old, list_new)
results["race_time_ms"] = pd.to_numeric(results["race_time_ms"])
results.to_csv("data_clean/results.csv")
results.dtypes

resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
race_time_ms       float64
dtype: object

### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [79]:
# Write your answer here
list_old = ['\\N']
list_new = [np.nan]
results["positionOrder"] = results["positionOrder"].replace(list_old, list_new)

results["positionOrder"] = pd.to_numeric(results["positionOrder"])

bins_x = [0,3,10,20,30]
labels_x = ["Podium",
            "Points",
            "Midfield",
            "Backmarker"]

results["finish_category"] = pd.cut(results["positionOrder"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)


### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [47]:
# Write your answer here
results["race_duration_minutes"] = results["race_time_ms"]/(1000*60)
constructor_agg = (results.groupby("constructorId")
                      .agg(mean_time = ('race_duration_minutes','mean')))
constructor_agg
race_duration_ascending = constructor_agg.sort_values(by = "mean_time", ascending = True)
display(race_duration_ascending.iloc[:5,:])


,mean_time
constructorId,
35,76.710777
29,77.604125
41,87.046767
16,89.428828
53,89.658852


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [48]:
# Write your answer here
driver_agg = (results.groupby(["driverId"])
                      .agg(avg_finishing_position = ('positionOrder','mean'),
                           total_points = ('points','sum'),
                           no_races_completed = ("raceId", len),
                           best_finishing_position = ('positionOrder', 'min')))

total_points_descending = driver_agg.sort_values(by = "total_points", ascending = False)
display(total_points_descending.iloc[:10,:])


,avg_finishing_position,total_points,no_races_completed,best_finishing_position
driverId,,,,
1,4.787097,4396.5,310,1
20,7.093333,3098.0,300,1
4,8.494413,2061.0,358,1
830,6.533742,1983.5,163,1
8,8.491477,1873.0,352,1
822,7.601990,1778.0,201,1
3,8.252427,1594.5,206,1
30,6.879870,1566.0,308,1
817,9.883621,1307.0,232,1


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [76]:
# Write your answer here
results[["grid","laps","points"]].dropna()

model = smf.ols(formula = 'points ~ grid + laps', data = results).fit(cov = "HC1")

print(model.summary())

predicted_points = model.predict(pd.DataFrame({'grid': [3], 'laps': [55]}))
print("predicted points:", predicted_points[0])

                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        15:57:25   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0